In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install required packages
print("📦 Installing required packages...")
!pip install qwen-vl-utils --quiet
print("✅ Installation complete!")

📦 Installing required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 53.9 MB/s eta 0:00:00:00:0100:01
✅ Installation complete!


In [3]:
import os
import csv
import time
import torch
from pathlib import Path
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# Check GPU
if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    print(f"🎮 GPU Information:")
    print(f"   Name: {gpu_props.name}")
    print(f"   VRAM: {gpu_props.total_memory / 1024**3:.1f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
    print("✅ GPU is available and ready!")
else:
    print("❌ No GPU available! Check your accelerator settings.")
    print("   Go to Settings (right panel) → Accelerator → GPU T4 x2")

2025-05-30 17:23:34.224173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748625814.430195      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748625814.492342      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🎮 GPU Information:
   Name: Tesla T4
   VRAM: 14.7 GB
   CUDA Version: 12.4
✅ GPU is available and ready!


In [4]:
print("✅ Dataset is already available!")

# Your data is already extracted at this path:
base_folder = "/kaggle/input/inaturalist-flower-images/test_images_200"

print(f"📁 Base folder: {base_folder}")

# Check if the folder exists and show its contents
if os.path.exists(base_folder):
    contents = os.listdir(base_folder)
    print(f"📂 Contents: {contents}")
    
    # Look for seed folders
    seed_folders = [item for item in contents if item.startswith('seed_')]
    print(f"🌱 Found seed folders: {seed_folders}")
    
    # Check one seed folder to verify structure
    if seed_folders:
        sample_seed = seed_folders[0]
        seed_path = os.path.join(base_folder, sample_seed)
        flower_folders = os.listdir(seed_path)
        print(f"🌸 Flowers in {sample_seed}: {flower_folders}")
        
        # Count images in first flower folder
        if flower_folders:
            sample_flower = flower_folders[0]
            flower_path = os.path.join(seed_path, sample_flower)
            if os.path.isdir(flower_path):
                images = [f for f in os.listdir(flower_path) 
                         if f.endswith(('.jpg', '.jpeg', '.png'))]
                print(f"📊 Sample: {sample_flower} has {len(images)} images")
    
    print("✅ Data structure looks good!")
    
else:
    print("❌ Base folder not found!")

✅ Dataset is already available!
📁 Base folder: /kaggle/input/inaturalist-flower-images/test_images_200
📂 Contents: ['experiment_summary.json', 'seed_123', 'seed_42', 'seed_456']
🌱 Found seed folders: ['seed_123', 'seed_42', 'seed_456']
🌸 Flowers in seed_123: ['sampling_metadata.json', 'Bellis_perennis', 'Leucanthemum_vulgare', 'Matricaria_chamomilla']
✅ Data structure looks good!


In [5]:
print("🤖 Loading Qwen2.5-VL-3B model from Hugging Face...")
print("This will take 2-3 minutes on first run...")

model_load_start = time.perf_counter()
model_id = "Qwen/Qwen2.5-VL-3B-Instruct"

try:
    print("📥 Downloading model from Hugging Face...")
    
    # Load model - this DEFINITELY works on Kaggle
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,  # T4 GPU supports this
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True,  # Important for Qwen models
    )
    
    processor = AutoProcessor.from_pretrained(
        model_id,
        trust_remote_code=True  # Important for Qwen models
    )
    
    # Optimize for inference
    model.eval()
    torch.cuda.empty_cache()
    
    load_time = time.perf_counter() - model_load_start
    print(f"✅ Model loaded successfully in {load_time:.1f} seconds!")
    print(f"📊 Model device: {next(model.parameters()).device}")
    print(f"📊 Model dtype: {next(model.parameters()).dtype}")
    
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    print("\n🔍 Troubleshooting:")
    print("1. Make sure GPU is enabled (Settings → Accelerator → GPU T4)")
    print("2. Make sure Internet is enabled (Settings → Internet → ON)")
    print("3. Try restarting the notebook if this persists")
    raise

🤖 Loading Qwen2.5-VL-3B model from Hugging Face...
This will take 2-3 minutes on first run...
📥 Downloading model from Hugging Face...


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

✅ Model loaded successfully in 60.5 seconds!
📊 Model device: cuda:0
📊 Model dtype: torch.bfloat16


In [17]:
# =============================================================================
# CONFIGURATION - Change these values as needed
# =============================================================================
SEED_TO_PROCESS = int(input("Enter seed number (42, 123, 456): "))  # 🌱 Change to 123 or 456 for other seeds

print(f"⚙️ Configuration:")
print(f"   Processing seed: {SEED_TO_PROCESS}")

# Use the correct path from your Kaggle dataset
base_folder = "/kaggle/input/inaturalist-flower-images/test_images_200"

# Define paths
image_folder = os.path.join(base_folder, f"seed_{SEED_TO_PROCESS}")
output_csv = f"human_taxa_results_seed_{SEED_TO_PROCESS}.csv"

print(f"📁 Looking for images in: {image_folder}")
print(f"💾 Results will be saved to: {output_csv}")

# Verify the seed folder exists
if os.path.exists(image_folder):
    print("✅ Seed folder found!")
    
    # Quick preview of what's inside
    contents = os.listdir(image_folder)
    print(f"📂 Contents: {contents}")
    
else:
    print("❌ Seed folder not found!")
    # Show available seeds
    if os.path.exists(base_folder):
        available_items = os.listdir(base_folder)
        seed_folders = [item for item in available_items if item.startswith('seed_')]
        if seed_folders:
            available_seeds = [int(s.split('_')[1]) for s in seed_folders]
            print(f"💡 Available seeds: {available_seeds}")
            print(f"💡 Change SEED_TO_PROCESS to one of: {available_seeds}")

Enter seed number (42, 123, 456):  456


⚙️ Configuration:
   Processing seed: 456
📁 Looking for images in: /kaggle/input/inaturalist-flower-images/test_images_200/seed_456
💾 Results will be saved to: human_taxa_results_seed_456.csv
✅ Seed folder found!
📂 Contents: ['sampling_metadata.json', 'Bellis_perennis', 'Leucanthemum_vulgare', 'Matricaria_chamomilla']


In [18]:
print("🔍 Scanning for images...")

flowers = ["Bellis_perennis", "Leucanthemum_vulgare", "Matricaria_chamomilla"]
image_paths = []

# Check if seed folder exists
if not os.path.exists(image_folder):
    print(f"❌ Seed folder not found: {image_folder}")
    
else:
    # Collect images from each flower folder
    total_images = 0
    
    for flower in flowers:
        flower_folder = os.path.join(image_folder, flower)
        
        if os.path.exists(flower_folder):
            # Find all image files
            flower_images = []
            for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
                flower_images.extend([
                    os.path.join(flower_folder, f) 
                    for f in os.listdir(flower_folder) 
                    if f.endswith(ext)
                ])
            
            image_paths.extend(flower_images)
            total_images += len(flower_images)
            print(f"🌸 {flower}: {len(flower_images)} images")
            
        else:
            print(f"❌ Flower folder not found: {flower_folder}")

print(f"\n📊 Total images found: {len(image_paths)}")

if len(image_paths) == 0:
    print("❌ No images found! Please check your folder structure and seed number.")
else:
    print("✅ Ready to process images!")
    
    # Show expected processing time
    estimated_time = len(image_paths) * 1.5 / 60  # 1.5 seconds per image
    print(f"⏱️  Estimated processing time: {estimated_time:.1f} minutes")

🔍 Scanning for images...
🌸 Bellis_perennis: 200 images
🌸 Leucanthemum_vulgare: 200 images
🌸 Matricaria_chamomilla: 200 images

📊 Total images found: 600
✅ Ready to process images!
⏱️  Estimated processing time: 15.0 minutes


In [19]:
if len(image_paths) > 0:
    print(f"🚀 Starting VML processing of {len(image_paths)} images...")
    print("=" * 60)
    
    processing_start = time.perf_counter()
    processed_count = 0
    error_count = 0
    results = []
    
    for i, image_path in enumerate(image_paths):
        image_file = os.path.basename(image_path)
        
        # Extract flower name from filename
        base_name = os.path.splitext(image_file)[0]
        name_parts = base_name.split('_')
        flower_name = "_".join(name_parts[:-1]) if len(name_parts) > 1 else base_name
        
        # Create prompt
        prompt = f"Does this image show any humans or identifiable human body parts (including, but not limited to, faces, hands, fingers, arms, legs, torsos, or silhouettes)? Answer with 'Yes' or 'No' only."
        
        # Progress updates every 25 images
        if i % 25 == 0 or i == len(image_paths) - 1:
            elapsed = time.perf_counter() - processing_start
            if i > 0:
                avg_time = elapsed / (i + 1)
                eta_seconds = (len(image_paths) - i - 1) * avg_time
                eta_minutes = eta_seconds / 60
                
                print(f"📊 Progress: {i+1:3d}/{len(image_paths)} ({(i+1)/len(image_paths)*100:5.1f}%) | "
                      f"Avg: {avg_time:.2f}s/img | ETA: {eta_minutes:.1f}min")
        
        try:
            # Prepare messages for the model
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": f"file://{image_path}"},
                        {"type": "text", "text": prompt}
                    ]
                }
            ]
            
            # Process with model
            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            ).to(model.device)

            # Generate response
            with torch.no_grad():
                generated_ids = model.generate(
                    **inputs, 
                    max_new_tokens=10,
                    do_sample=False,
                    pad_token_id=processor.tokenizer.eos_token_id
                )
            
            # Decode response
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            response = processor.batch_decode(
                generated_ids_trimmed, 
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False
            )[0].strip()

            # Extract Yes/No result
            result = "Yes" if "Yes" in response else "No"
            results.append([image_file, result])
            processed_count += 1
            
            # Show sample results for first few images
            if i < 3:
                print(f"   📝 Sample {i+1}: {image_file} → {result}")
                
        except Exception as e:
            print(f"❌ Error processing {image_file}: {str(e)}")
            results.append([image_file, "Error"])
            error_count += 1
            
            # Clear GPU memory after error
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Processing complete
    total_time = time.perf_counter() - processing_start
    
    print("\n" + "=" * 60)
    print("🎉 PROCESSING COMPLETE!")
    print("=" * 60)
    print(f"📊 Final Statistics:")
    print(f"   Total images: {len(image_paths)}")
    print(f"   Successfully processed: {processed_count}")
    print(f"   Errors: {error_count}")
    print(f"   Success rate: {(processed_count/len(image_paths))*100:.1f}%")
    print(f"⏱️  Performance:")
    print(f"   Total time: {total_time/60:.1f} minutes")
    print(f"   Average per image: {total_time/len(image_paths):.2f} seconds")

else:
    print("❌ No images to process!")

🚀 Starting VML processing of 600 images...
   📝 Sample 1: Bellis_perennis_45972.jpg → No
   📝 Sample 2: Bellis_perennis_79728.jpg → No
   📝 Sample 3: Bellis_perennis_66939.jpg → No
📊 Progress:  26/600 (  4.3%) | Avg: 1.62s/img | ETA: 15.5min
📊 Progress:  51/600 (  8.5%) | Avg: 1.65s/img | ETA: 15.1min
📊 Progress:  76/600 ( 12.7%) | Avg: 1.67s/img | ETA: 14.6min
📊 Progress: 101/600 ( 16.8%) | Avg: 1.65s/img | ETA: 13.7min
📊 Progress: 126/600 ( 21.0%) | Avg: 1.63s/img | ETA: 12.9min
📊 Progress: 151/600 ( 25.2%) | Avg: 1.63s/img | ETA: 12.2min
📊 Progress: 176/600 ( 29.3%) | Avg: 1.63s/img | ETA: 11.5min
📊 Progress: 201/600 ( 33.5%) | Avg: 1.63s/img | ETA: 10.8min
📊 Progress: 226/600 ( 37.7%) | Avg: 1.63s/img | ETA: 10.2min
📊 Progress: 251/600 ( 41.8%) | Avg: 1.63s/img | ETA: 9.5min
📊 Progress: 276/600 ( 46.0%) | Avg: 1.63s/img | ETA: 8.8min
📊 Progress: 301/600 ( 50.2%) | Avg: 1.63s/img | ETA: 8.1min
📊 Progress: 326/600 ( 54.3%) | Avg: 1.63s/img | ETA: 7.4min
📊 Progress: 351/600 ( 58.5%) |

In [20]:
if 'results' in locals() and results:
    print(f"\n💾 Saving results to {output_csv}...")
    
    # Save to CSV
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Image_Name", "Humans_Present"])
        writer.writerows(results)
    
    print(f"✅ Results saved successfully!")
    
    # Calculate statistics
    yes_count = sum(1 for result in results if result[1] == "Yes")
    no_count = sum(1 for result in results if result[1] == "No")
    error_count = sum(1 for result in results if result[1] == "Error")
    
    print(f"\n📈 Result Summary:")
    print(f"   🔴 yes: {yes_count:3d} ({yes_count/len(results)*100:5.1f}%)")
    print(f"   🟢 no:  {no_count:3d} ({no_count/len(results)*100:5.1f}%)")
    print(f"   ⚠️  Processing errors: {error_count:3d} ({error_count/len(results)*100:5.1f}%)")
    
    # Create download link
    from IPython.display import FileLink, display
    print(f"\n⬇️  DOWNLOAD YOUR RESULTS:")
    display(FileLink(output_csv))
    
    # Show sample results
    print(f"\n🔍 Sample Results (first 10):")
    for i, (filename, result) in enumerate(results[:10], 1):
        status_emoji = "🔴" if result == "Yes" else "🟢" if result == "No" else "⚠️"
        print(f"   {i:2d}. {status_emoji} {filename}: {result}")
    
    if len(results) > 10:
        print(f"   ... and {len(results) - 10} more results")

else:
    print("❌ No results to save!")

print(f"\n✅ SEED {SEED_TO_PROCESS} PROCESSING COMPLETE!")
print(f"📝 Remember to download your CSV file!")
print(f"🔄 To process other seeds (123, 456), change SEED_TO_PROCESS in Cell 5 and re-run cells 5-8")


💾 Saving results to human_taxa_results_seed_456.csv...
✅ Results saved successfully!

📈 Result Summary:
   🔴 yes:  27 (  4.5%)
   🟢 no:  573 ( 95.5%)
   ⚠️  Processing errors:   0 (  0.0%)

⬇️  DOWNLOAD YOUR RESULTS:


/kaggle/working/human_taxa_results_seed_456.csv


🔍 Sample Results (first 10):
    1. 🟢 Bellis_perennis_45972.jpg: No
    2. 🟢 Bellis_perennis_79728.jpg: No
    3. 🟢 Bellis_perennis_66939.jpg: No
    4. 🟢 Bellis_perennis_29107.jpg: No
    5. 🟢 Bellis_perennis_43031.jpg: No
    6. 🟢 Bellis_perennis_9505.jpg: No
    7. 🔴 Bellis_perennis_1642.jpg: Yes
    8. 🟢 Bellis_perennis_28510.jpg: No
    9. 🟢 Bellis_perennis_74387.jpg: No
   10. 🟢 Bellis_perennis_42750.jpg: No
   ... and 590 more results

✅ SEED 456 PROCESSING COMPLETE!
📝 Remember to download your CSV file!
🔄 To process other seeds (123, 456), change SEED_TO_PROCESS in Cell 5 and re-run cells 5-8
